In [ ]:
import sqlite3

In [ ]:
sqlite3

<module 'sqlite3' from '/usr/lib/python3.10/sqlite3/__init__.py'>

1python --> 2DBMS --> 3DB
         (SQLite)

package sqlite3 func(SQL)
아무짓도 안함. data 이미 db 가정 -> 3
python -> select -> sqlite db로 가져옴 --> 2 (cursor)
          fetch()

In [ ]:
# DBMS == SERVER
# 나는 DBMS의 클라이언트이다.
# 원래는 커넥션을 생성한다. 그런데 SQL 라이트는 파일이 열러있있으면 커넥션이 되어 이쓴 것

In [ ]:
# 확장자는 그냥 사람이 파일의 형식을 파악하기 위함이다.

# 1.커넥션 만들기
# sqlite는 서버 url 대신에 파일경로 또는 메모리 경로를 인자로 넣어준다.
conn = sqlite3.connect('test.db')

In [ ]:
# 2. cursor 만들기, 이것이 데이터베이스에 관한 모든 일을 한다(?)
cur = conn.cursor()

In [ ]:
# 3. 데이터베이스 연결 끊기
conn.close()

In [ ]:
!ls

sample_data  test.db


In [ ]:
conn = sqlite3.connect('test.db')

In [ ]:
cur = conn.cursor()

In [ ]:
cur.execute() # SQL 한번 실행
cur.executemany() # 동일 SQL 여러번 실행
cur.executescript() # 여러 SQL 동시 실행 -> 비표준 사용 지양하기

In [ ]:
cur.execute('''
  CREATE TABLE CITY(
    PK INTEGER PRIMARY KEY,
    NAME VARCHAR(1)
  );
''')

In [ ]:
cur.execute('''
  INSERT INTO CITY (PK, NAME) VALUES (1, "안암점");
''')

In [ ]:
gu = '성북구'
# 큐마크 스타일
cur.execute('''
  INSERT INTO CITY (NAME) VALUES (?)
''', (gu,))

In [ ]:
gu = '성북구'
# 네임드 스타일
cur.execute('''
  INSERT INTO CITY (NAME) VALUES (:key)
''', {'key':gu})

In [ ]:
cur.executemany('''
  INSERT INTO CITY (NAME) VALUES (?)
''',(('노원구',), ('강북구',), ('중구',)))

큐마크와 네임드는 파이썬의 Obj를  DBMS에 전달하는 방법을 배운것

In [ ]:
cur.lastrowid

3

In [ ]:
cur.execute('SELECT * FROM CITY')

In [ ]:
# fetch 커서에서 작업해서 임시로 들고 있는 것을 파이썬으로 가져오는 역할
city = cur.fetchall()
# cur.fetchone()는 city[4]와 같다
# cur.fetchmany()는 city[:4]와 같다

In [ ]:
type(city), len(city)

(list, 6)

In [ ]:
city

[(1, '안암점'), (2, '성북구'), (3, '성북구'), (4, '노원구'), (5, '강북구'), (6, '중구')]

In [ ]:
conn.close()

In [ ]:
conn = sqlite3.connect('test.db')
cur = conn.cursor()

In [ ]:
cur.execute('SELECT * FROM CITY')

In [ ]:
cur.fetchall() # 닫았다가 열었더니, 데이터가 받아와 지지 않는다.

[(1, '성북구'), (2, '강북구')]

In [ ]:
# SQLite에서는 데이터 형이 적다. 문자열 TEXT 밖에 없고 길이 제한 효과 없다

In [ ]:
# 데이터의 안정성이 확보 되었을 때, 데이터베이스에 변화를 반영하는 작업이다. -> 이 커밋되는 단위를 트랜젝션이라고 부른다.
conn.commit()

In [ ]:
cur.executescript('''
  DROP TABLE IF EXISTS SUPPLIER;
  CREATE TABLE SUPPLIER(
    PK INTEGER PRIAMRY KEY,
    NAME TEXT,
    FK INTEGER NOT NULL
  );
  DROP TABLE IF EXISTS PART;
  CREATE TABLE PART (
    PK INTEGER PRIMARY KEY,
    NAME TEXT
  );
  DROP TABLE IF EXISTS SELLS;
  CREATE TABLE SELLS (
    FK1 INTEGER NOT NULL,
    FK2 INTEGER NOT NULL
  );
''')

OperationalError: database is locked

In [ ]:
# Qmark
cur.executemany('''
  INSERT INTO CITY (PK, NAME) VALUES (?,?)
''', ((1,'강북구'),(2,'성북구'),(3,'중구'),))

In [ ]:
# Named
cur.executemany('''
  INSERT INTO PART VALUES(:no, :name)
''',({'no':1, 'name':'아메리카노'},{'no':2, 'name':'카페라때'},{'no':3, 'name':'바닐라라떼'},{'no':4, 'name':'아이스티'}))

In [ ]:
conn.commit()

In [ ]:
conn.close()

In [ ]:
conn = sqlite3.connect('test.db')
cur = conn.cursor()

In [ ]:
cur.execute("SELECT * FROM CITY, PART")

In [ ]:
cur.fetchall()

[(1, '성북구', 1, '아메리카노'),
 (1, '성북구', 2, '카페라때'),
 (1, '성북구', 3, '바닐라라떼'),
 (1, '성북구', 4, '아이스티'),
 (2, '강북구', 1, '아메리카노'),
 (2, '강북구', 2, '카페라때'),
 (2, '강북구', 3, '바닐라라떼'),
 (2, '강북구', 4, '아이스티')]

In [ ]:
# SUPPLIER
# cur.execute('INSERT INTO SUPPLIER VALUES(1,"안암점",?)')
gu = '성북'
cur.execute('''
  SELECT PK
  FROM CITY
  WHERE NAME LIKE ?
  ORDER BY PK ASC LIMIT 0, 1
''',('%'+gu+'%',))  # '''LIMIT 0,1''' 0번째 위치에서 1개
rst = cur.fetchone()
fk = None if rst == None else rst[0]

In [ ]:
store = '안암점'
if fk is not None:
  cur.execute('''
    INSERT INTO SUPPLIER VALUES(NULL,?,?);
  ''',(store, fk))

OperationalError: database is locked

In [ ]:
# 중첩 SQL
cur.execute('''
  INSERT INTO SUPPLIER VALUES(NULL,?,(
    SELECT PK
    FROM CITY
    WHERE NAME LIKE ?
    ORDER BY PK ASC LIMIT 0, 1
  ));
''',('종암점', '%성북%'))

OperationalError: database is locked

In [ ]:
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[]

In [ ]:
conn.close()

In [ ]:
conn = sqlite3.connect('test.db')
cur = conn.cursor()

In [ ]:
cur

In [ ]:
cur.execute('SELECT * FROM SUPPLIER;')

In [ ]:
# 놓쳤다~~~

[]

In [ ]:
# CITY - SUPLLIER
cur.execute('''
  SELECT B.NAME, A.NAME FORM SUPPLIER AS A
  LEFT JOIN CIRY AS B
  ON B.PK = A.FK
  ORDER BY B.NAME ASC
''')
cur.fetchall()

OperationalError: near "SUPPLIER": syntax error

In [ ]:
# CITY - SUPLLIER
cur.execute('''
  SELECT B.NAME, A.NAME FORM SUPPLIER AS A
  LEFT JOIN CIRY AS B
  ON B.PK = A.FK
  ORDER BY B.NAME ASC
''')
cur.fetchall()

In [ ]:
# DB Browser for SQLite
# try except - rollback, commit, transaction

# 질문
# import한 sqlite3 패키지는 클라이언트 API 라고 해도 되는가?

# 내일 추천 노래 리스트를 뽑아주는 SQL로직을 짜볼것이다.

# 류기곤 코치님께서 엔티티를 찾을 때,
# - 각 개념별 종속 관계를 먼저 파악하셨고,
# - 그 다음에는 얻어낸 엔티티와 나머지 속성들과의 카디널리디(중복의 정도)를 파악해서 엔티티 속성에 포함시킬지 다른 하나의 엔티티로 분리할지 결정하셨다.